In [1]:
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score 
import fastapi
from typing import Optional

In [2]:
_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"] 
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [3]:
def model_evaluation(y_test, y_pred) :
    y_test = [x == 'True' if isinstance(x, str) else x for x in y_test]
    y_pred = [x == 'True' if isinstance(x, str) else x for x in y_pred]
    
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred)
    testprecision=precision_score(y_test, y_pred)
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    return testacc, testrecall, testprecision

In [4]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [5]:
def retrain_model(last_n: int, model_type: str = "Dummy", strategy: Optional[str] = None, k: Optional[int] = 3):
    global model

    if len(_x) < last_n or len(_result) < last_n:
        return {"error": "Not enough data to retrain the model."}

    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])

    if model_type == "Dummy":
        model = DummyClassifier(strategy=strategy or "stratified")
    elif model_type == "KNN":
        model = KNeighborsClassifier(n_neighbors=k)
    else:
        return {"error": "Unsupported model type."}

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, stratify=Y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {
        "message": f"Model retrained using {model_type}",
        "accuracy": testacc,
        "recall": testrecall,
        "precision": testprecision
    }

In [6]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

@app.get("/retrain/")
def retrain(last_n: int, model_type: str = "Dummy", strategy: Optional[str] = "stratified", k: Optional[int] = 3):
    return retrain_model(last_n=last_n, model_type=model_type, strategy=strategy, k=k)

@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [7]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
{'message': "Model trained DummyClassifier(strategy='stratified')", 'train': {'error': 'Not enough data to retrain the model.'}}


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'error': 'Not enough data to retrain the model.'}}

In [8]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Started server process [17904]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1 -0.8065275407319747 -0.06940842655876422
True
INFO:     127.0.0.1:57392 - "GET /predict/?id=1&x=-0.8065275407319747&y=-0.06940842655876422 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57393 - "GET /result/?id=1&result=True HTTP/1.1" 200 OK
2 0.7683659989915474 0.7421044814085227
True
INFO:     127.0.0.1:57394 - "GET /predict/?id=2&x=0.7683659989915474&y=0.7421044814085227 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57395 - "GET /result/?id=2&result=True HTTP/1.1" 200 OK
3 0.9346419449569874 0.799136062259123
True
INFO:     127.0.0.1:57396 - "GET /predict/?id=3&x=0.9346419449569874&y=0.799136062259123 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57397 - "GET /result/?id=3&result=True HTTP/1.1" 200 OK
4 0.8039224443718216 -0.7323586859230509
True
INFO:     127.0.0.1:57398 - "GET /predict/?id=4&x=0.8039224443718216&y=-0.7323586859230509 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:57399 - "GET /result/?id=4&result=False HTTP/1.1" 200 OK
5 -0.3253115361852654 0.6098510253366816
True
INFO:     127.